# Introduction to the _mip_ Python module

The module we'll become most familiar with in this course is the `mip` module, which allows for creating, manipulating, and solving optimization models with linear constraints and integer, binary, or continuous variables. Check the [homepage](https://www.python-mip.com) for full access to the documentation and updates.

Suppose you want to model the following problem:
$$
\begin{array}{ll}
  \max & x_1 + x_2\\
  \textrm{s.t.} & 2 x_1 + x_2 \le 10\\
  & x_1, x_2 \ge 0
\end{array}
$$

For starters, we import the module `mip` in python.

In [1]:
# When using Colab, make sure you run this instruction beforehand
# Python tools for the modeling and solution of Mixed-Integer Linear programs = MIP
!pip install mip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 126.7 kB/s eta 0:00:0000:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.0/179.0 kB 100.8 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: cffi
    Found existing installation: cffi 1.14.6
    Uninstalling cffi-1.14.6:
      Successfully uninstalled cffi-1.14.6


In [3]:
# Import module
# un file python dobbiamo dirlo esplicitamente che lo importiamo
import mip

Next, we create an optimization model `m`. We do so by calling the `mip.Model` *constructor* method. We also create two variables `x1` and `x2` using the `add_var()` method from the optimization model.

In [17]:
# Create model and add two variables to it
# Creazione modello di ottimizzazione mip.Model() -> aggiungo variabili

m = mip.Model();
x1 = m.add_var();
x2 = m.add_var();

We now add the single constraint and the objective. 

To add the constraint, we use the method `add_constr` from the optimization model.
To add the objective function, we set the `objective` attribute of `m`. We use the method `mip.maximize`, to indicate that this is a function to be maximized.

For now, since both the constraint and the objective are very simple, we fully write them as algebraic expressions of `x1` and `x2`.

In [18]:
# Add constraint, set objective function
m.add_constr(2*x1+x2<=10)
m.objective= mip.maximize(x1+x2) # funzione obiettivo

Finally, we call the method `optimize` to solve the problem and print the value of the optimal solution. For a variable `v` of the module `mip`, its value in the optimal solution is retrieved as the attribute `.x`, for example `v.x`.

In [22]:
# optimize and print solution
m.optimize() # la soluzione viene 0 -> Ottimizza il modello (con funz. ob. e vincoli)
print(x1.x,x2.x) # per il valore dobbiamo mettere .x

0.0 10.0


# A slightly more advanced example

Let us now consider a slightly more complicated example: formulating and solving a knapsack problem.

$$
\begin{array}{lll}
\max & 3 x_1 + 4 x_2 + 7 x_3 + 5 x_4\\
\textrm{s.t.} & 4 x_1 + 5 x_2 + 6 x_3 + 4 x_4 \le 13\\
              & x_1, x_2, x_3, x_4 \in \{0,1\}
\end{array}
$$

For starters, we import the module and define the data used in this model.

In [23]:
# First: import the Python-MIP module
import mip 

Next, we create an optimization model with the `mip.Model` *constructor* method. 

We also add four variables using a list, and call that list `x`. Note that we are using a so-called _list comprehension_ to create variables, i.e., we put a `for` construct _inside_ the list in order to create as many list elements as there are numbers in `range(4)`. As you may have gathered from previous cells, `range(4)` is the set of numbers `0, 1, 2, 3`.

In [28]:
# Create model and variables

valore = [3, 4, 7, 5]
peso = [4, 5, 6, 4]
capacita = 13

m=mip.Model();

# x1=m.add_var(var_type=mip.BINARY)
# x2=m.add_var(var_type=mip.BINARY)
# x3=m.add_var(var_type=mip.BINARY)
# x4=m.add_var(var_type=mip.BINARY)

#x= (m.add_var(var_type=mip.BINARY),
#   m.add_var(var_type=mip.BINARY),
#   m.add_var(var_type=mip.BINARY),
#   m.add_var(var_type=mip.BINARY))

# Creare una lista di variabili con add_var e dico di farlo per tutti i numeri da 0 a 3
x= [m.add_var(var_type=mip.BINARY) for i in range(4)]
print(x)

[<mip.entities.Var object at 0x7fd6c91e2970>, <mip.entities.Var object at 0x7fd6c91e29d0>, <mip.entities.Var object at 0x7fd6c91e2af0>, <mip.entities.Var object at 0x7fd6c91e2670>]


We now add the single constraint and the objective. In order to create the sum $\sum_i w_i x_i$, the method `mip.xsum` houls be used. As an argument, one again uses a `for` construct inside the `xsum` argument. The expression

```python
weight[i] * x[i] for i in range(4)
```

generates all products $w_ix_i$ for all $i\in \{0,1,2,3\}$ (I know it might be tricky for many to get used to the idea that indices begin at zero in Python, but this will come in handy in the future). This expression is then wrapped inside a `mip.xsum`, which is constrained to be lesser than or equal to `max_weight`. This is the constraint. It is added to the model with the `+=` operator, which is common in Python and other languages such as C/C++ or Java; `a += b` means "add `b` to `a` and store the result in `a`".

The objective function is a similar `mip.xsum` construction, this time with `value[i]` instead of `weight[i]` for coefficients. It is assigned as the model's objective function with the method `mip.maximize`, to indicate that this is obviously a function to be maximized.

In [34]:
# Add constraint, set objective
# m.add_constr(somma di tutti i ... per i che va da uno a quattro  <= capacità zaino)
m.add_constr(mip.xsum(peso[i]*x[i] for i in range(4)) <= 13)
# Lista indicizzabile da 0 a N-1
m.objective = mip.maximize(mip.xsum(valore[i]*x[i] for i in range(4)))

Finally, we call the method `optimize` to solve the problem and print the value of the optimal solution. For a variable `v` of the module `mip`, its value in the optimal solution is retrieved as the attribute `.x`, for example `v.x`.

In [35]:
# Optimize and print solution
m.optimize()
print(x[0].x,x[1].x,x[2].x,x[3].x)

Coin0506I Presolve 0 (-1) rows, 0 (-2) columns and 0 (-2) elements
Clp0000I Optimal - objective value 10
Coin0511I After Postsolve, objective 10, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 10 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00
Clp0000I Optimal - objective value 10
Clp0000I Optimal - objective value 10
Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 1 (-2) rows, 4 (0) columns and 4 (-8) elements
Clp1000I sum of infeasibilities 0 - average 0, 4 fixed columns
Coin0506I Presolve 0 (-1) rows, 0 (-4) columns and 0 (-4) elements
Clp0000I Optimal - objective value -0
Clp0000I Optimal - objective value -0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj -0 Dual inf 23.799996 (4)
Clp0000I Optimal - objective value 14.4
Clp0000I Optimal - objective value 14.4
Clp0000I Optimal - objective value 14.4
Coin0511I After Postsolve, objective 14.4, infeasi

Obviously, we could have just formulated the non-parametric version of the problem. Here it is:

In [ ]:
# Write problem in non-parametric form.

## Miscellanea and troubleshooting

After this first MIP model it's time to say something more about Python.

### Re-running code on Jupyter notebooks
Code on Jupyter notebooks is fed into Python one cell at a time. If the notebook is written correctly, you should be able to click into the first cell, then just do a `shift`+`enter` through the last cell without any error.

You are also able to re-run any cell multiple times, in any sequence you want. However, be aware that Python sees a sequence of cells it is given, and does not know whether an instruction should be undone or not. Therefore, once a cell is run, its results are _persistent_, at least until we reset them. One big red button is the __Restart__ command under the _Kernel_ tab in the menu: it clears all memory of whatever was done in the cell so far (though obviously not file operations). Later in this notebook we show an example of the trouble persistence can cause.

### Indentation
Indentation is crucial: in a `for` loop, an `if` block, or a function definition, the inner part __MUST__ be indented consistently. Python will throw an error in the following cases:

```python
for i in [1,2,3]:
print(i)
```
Here the `print` instruction should be indented by at least one space.
```python
if i==4:
    print('i is 4')
  print('deal with it')
```
Here indentation is inconsistent.
```python
def myfunction(a):
return a**4 + 5*a**3
```
Same as the first incorrect example. The correct way to write these examples is as follows:
```python
for i in [1,2,3]:
    print(i)

if i==4:
    print('i is 4')
    print('deal with it')

def myfunction(a):
    return a**4 + 5*a**3
```
The suggested indentation is 4 characters.

### Assignment vs. equality
The sign `=` is for _assignment_, while `==` is for checking equality of two expressions. You can write `if a == 4` but not `if a = 4`. Also, writing the statement `a = 4` is correct, and so is `a == 4`; however, the latter has no effect (apart from returning `True` or `False` on the Python command line).

### Semicolons, be gone!
You may have noticed that Python doesn't require semicolons (`;`) at the end of each instruction, as other languages like C, C++, Java, AMPL do. This makes for more readable and prettier code, but indentation is enforced with this in mind.

### Writing a statement on multiple lines
Related to the last point: conditions can be split on multiple lines as long as a `\` is added at the end of all but the last one, for example:
```python
if i==3 or \
   i==4:
    print('i is not 5')
```
But the `\` is not necessary if there is an unclosed parenthesis, for instance:
```python
if (i==3 or i==5 or
    i==7):
    print('i is prime')
```
### If you're feeling a bit masochistic...
A good way to check if your Python program was written according to the standard is to run the `flake8` module on it. Just run `flake8 myprogram.py` and check all the errors it throws (there are usually a ton).

## Persistence and debugging in Jupyer notebooks

Suppose you want to model the following problem:
$$
\begin{array}{ll}
  \max & x_1 + x_2\\
  \textrm{s.t.} & 2 x_1 + x_2 \le 10\\
  & x_1, x_2 \ge 0
\end{array}
$$
Let's write the model using `mip`:

In [ ]:
# Copy full model here

Suppose now we want to relax the constraint, for instance change the right-hand side to 20:

In [ ]:
# Add relaxed constraints (e.g. with <= 20 instead of <= 10)

# Re-optimize and print the solution.

print('solution:', x1.x, ',', x2.x)

The solution is the same even though we relaxed the problem. Why? Well, the problem has two constraints: the one we added in the first cell (which is the more restrictive one) and the last constraint. If we want to relax a problem or change it otherwise, we should modify the cell it is contained in.